In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
!tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
!git clone -q --depth 1 https://github.com/CMU-Perceptual-Computing-Lab/openpose.git
!sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
!apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
!pip3 install -q youtube-dl
!cd openpose && rm -rf build || true && mkdir build && cd build && cmake -DBUILD_PYTHON=ON .. && make -j`nproc`

In [ ]:
!pip3 install -r /content/drive/MyDrive/TCC/Tracker/requirements.txt

In [ ]:
!pip3 install fastapi nest-asyncio pyngrok uvicorn python-multipart

In [ ]:
from tqdm import tqdm
from fastapi import FastAPI, File, UploadFile
from typing import List
from pyngrok import ngrok
from datetime import datetime

import os
import csv
import json
import copy
import shutil
import math
import uvicorn
import nest_asyncio
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
colab_path = '/content/drive/MyDrive/TCC'

xh, yh = 1, 0
threshold = -1

neck = 1
hip = 8
joints = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
limbs = [(2, 3), (3, 4), (5, 6), (6, 7), (9, 10), (10, 11), (12, 13), (13, 14)]

total_bins = 16

In [ ]:
def euclidean_1d(p1, p2):
    distance = 0

    for i in range(len(p1)):
        distance = distance + (float(p1[i]) - float(p2[i])) ** 2

    return distance ** 0.5

In [ ]:
def save_files(files, path):
    os.mkdir(path)
    
    for image in files:
        with open(path + '/' + image.filename, 'wb') as buffer:
            shutil.copyfileobj(image.file, buffer)

In [ ]:
def detect_poses(in_path):
    get_ipython().system('cd openpose && ./build/examples/openpose/openpose.bin --image_dir ' + in_path + ' --write_json ' + path_estimation + ' --display 0 --render_pose 0')

In [ ]:
def track_poses(in_path):
    a = 0

In [ ]:
def association(subjects, sequence):
    path_boundingbox = os.path.join(path_multi_boundingboxes, os.path.join(subjects, sequence))
    path_keypoints = os.path.join(path_multi_keypoints, os.path.join(subjects, sequence))

    with open(os.path.join(path_boundingbox, 'bbox.csv'), 'r') as f:
        reader = csv.reader(f)
        boundingboxes_list = list(reader)
        
    ids = []

    frames = []
    num_frames = len(os.listdir(path_keypoints))

    boundingboxes = [[] for i in range(num_frames)]

    for boundingbox in boundingboxes_list:
        frame = int(boundingbox[0])

        if frame >= num_frames:
            continue

        id = int(float(boundingbox[1]))
        ids.append(id)

        x1, y1, x2, y2 = float(boundingbox[2]), float(boundingbox[3]), float(boundingbox[4]), float(boundingbox[5])
        confbox = float(boundingbox[6])

        boundingboxes[frame].append([id, x1, y1, x2, y2, confbox])
        frames.append(frame)

    ids = set(ids)
    num_ids = len(ids)
    indexes = {}

    for i, id in enumerate(ids):
        indexes[id] = i

    frames = set(frames)
    
    subjects_keypoints = [[] for i in range(num_ids)]
    
    for frame in frames:
        frame_string = str(frame)
        keypoints_file = subjects + '_' + sequence + '_' + frame_string.zfill(12) + '.json'

        try:
            path = os.path.join(path_multi_keypoints, os.path.join(subjects, sequence))

            with open(os.path.join(path, keypoints_file), 'r') as f:
                keypoints_json = json.load(f)
                keypoints_list = keypoints_json['people']
        except FileNotFoundError:
            continue

        num_boundingboxes = len(boundingboxes[frame])
        boundingbox_used = [False for i in range(num_boundingboxes)]

        for i, keypoints in enumerate(keypoints_list):
            distances = []

            for j, boundingbox in enumerate(boundingboxes[frame]):
                if boundingbox_used[j] == True:
                    continue
                    
                id = int(boundingbox[0])
                x1, y1, x2, y2 = float(boundingbox[1]), float(boundingbox[2]), float(boundingbox[3]), float(boundingbox[4])
                x3, y3, x4, y4 = x1, y2, x2, y1
                confbox = float(boundingbox[5])

                key = keypoints['keypoints']

                for joint in joints:
                    x, y = float(key[3 * joint]), float(key[3 * joint + 1])
                    conf = float(key[3 * joint + 2])

                    if (x == 0. and y == 0.):
                        distances.append((float('inf'), -1, -1))
                    else:
                        d1 = euclidean_1d([x, y], [x1, y1])
                        d2 = euclidean_1d([x, y], [x2, y2])
                        d3 = euclidean_1d([x, y], [x3, y3])
                        d4 = euclidean_1d([x, y], [x4, y4])
                        distances.append((max(d1, d2, d3, d4), id, j))

            distances = sorted(distances)
            
            if len(distances) == 0 or distances[0][0] == float('inf'):
                continue

            id = distances[0][1]
            subjects_keypoints[indexes[id]].append(keypoints['keypoints'])
            
            boundingbox_used[distances[0][2]] = True
    
    return subjects_keypoints

In [ ]:
def extract_features(keypoints_list):
    angles_list = []
    distances_list = []
    anthropometrics_list = []
    
    for limb in limbs:
        angles = []
        distances = []
        anthropometrics = []

        for keypoints in keypoints_list:
            x1, y1 = float(keypoints[3 * limb[0]]), float(keypoints[3 * limb[0] + 1])
            conf1 = float(keypoints[3 * limb[0] + 2])
            
            x2, y2 = float(keypoints[3 * limb[1]]), float(keypoints[3 * limb[1] + 1])
            conf2 = float(keypoints[3 * limb[1] + 2])
            
            xw, yw = x2 - x1, y2 - y1
            
            xneck, yneck = float(keypoints[3 * neck]), float(keypoints[3 * neck + 1])
            confneck = float(keypoints[3 * neck + 2])
            
            xhip, yhip = float(keypoints[3 * hip]), float(keypoints[3 * hip + 1])
            confhip = float(keypoints[3 * hip + 2])
                
            neck_to_hip = euclidean_1d([xneck, yneck], [xhip, yhip])

            if (x1 == 0 and y1 == 0) or (x2 == 0 and y2 == 0) or (xw == 0 and yw == 0):
                rho = -1
            else:
                rho = math.acos((xw * xh + yw * yh) / (((xw * xw + yw * yw) ** 0.5) * ((xh * xh + yh * yh) ** 0.5)))

            if (x1 == 0 and y1 == 0) or (x2 == 0 and y2 == 0) or (xw == 0 and yw == 0) or (xneck == 0 and yneck == 0):
                norm = 0
            else:
                xv, yv = xneck - x1, yneck - y1
                scalar = (xw * xv + yw * yv) / (((xw * xw + yw * yw) ** 0.5) ** 2)
                projx, projy = xw * scalar, yw * scalar
                dx, dy = xv - projx, yv - projy
                norm = (dx * dx + dy * dy) ** 0.5

            if (x1 == 0 and y1 == 0) or (x2 == 0 and y2 == 0) or neck_to_hip == 0:
                dist = 0
            else:
                dist = euclidean_1d([x1, y1], [x2, y2]) / neck_to_hip

            angles.append(rho)
            distances.append(math.log2(norm + 1))
            anthropometrics.append(dist)

        angles_list.append(angles)
        distances_list.append(distances)
        anthropometrics_list.append(anthropometrics)

    return angles_list, distances_list, anthropometrics_list

In [ ]:
def create_histograms(angles, distances, anthropometrics):
    with open(path_single_distance_max, 'r') as f:
        max_distance = float(f.read())

    with open(path_single_anthropometrics_max, 'r') as f:
        max_anthropometrics = float(f.read())

    histogram_angles, bins, patches = plt.hist(angles, total_bins, range=(-1, math.pi))
    histogram_distances, bins, patches = plt.hist(distances, total_bins, range=(0, math.log2(max_distance + 1)))
    histogram_anthropometrics, bins, patches = plt.hist(anthropometrics, total_bins, range=(0, max_anthropometrics))

    for arr in range(len(histogram_angles)):
        for val in range(len(histogram_angles[arr])):
            if len(angles[arr]) != 0:
                histogram_angles[arr][val] = histogram_angles[arr][val] / len(angles[arr])

    for arr in range(len(histogram_distances)):
        for val in range(len(histogram_distances[arr])):
            if len(distances[arr]) != 0:
                histogram_distances[arr][val] = histogram_distances[arr][val] / len(distances[arr])

    for arr in range(len(histogram_anthropometrics)):
        for val in range(len(histogram_anthropometrics[arr])):
            if len(anthropometrics[arr]) != 0:
                histogram_anthropometrics[arr][val] = histogram_anthropometrics[arr][val] / len(anthropometrics[arr])
                
    return histogram_angles, histogram_distances, histogram_anthropometrics

In [ ]:
class HTTPServer():
    def __init__(self, app=FastAPI()):
        self.app = app
        
        @app.get('/index')
        async def home():
            return 'Hello World'

        @app.post('/extractfeatures/')
        async def create_upload_files(files: List[UploadFile] = File(...)):
            now = datetime.now()
            folder = now.strftime('%H-%M-%S')
            response = list()
            
            try:
                save_files(files=files, path=path_input)
                
                detect_poses(in_path=path_input,out_path=path_output_keypoints)
                track_poses(in_path=path_input, out_path=path_output_boudingboxes)
                
                keypoints_list = association()
                
                for keypoints_list in subject_keypoints:
                    angles, distances, anthropometrics, max_d, max_l = extract_features(keypoints_list)
                    hist_angles, hist_dist, hist_lens = create_histograms(angles, distances, anthropometrics)
                    response.append({'angles': hist_angles.tolist(), 'distances': hist_dist.tolist(), 'anthropometrics': hist_lens.tolist()})
            finally:
                get_ipython().system('rm -rf '+ path_input)
                
            return response

    def run(self, port=8000):
        ngrok_tunnel = ngrok.connect(port)
        nest_asyncio.apply()
        uvicorn.run(self.app, port=port)
        
        print('Public URL:', ngrok_tunnel.public_url)

In [ ]:
app = HTTPServer()
app.run()